In [1]:
pip install nltk

In [2]:
# importing required libraries and text processing units
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import ast
import gensim.downloader as api

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
#model = api.load("glove-wiki-gigaword-300")
model = api.load("fasttext-wiki-news-subwords-300")

[==================================================] 100.0% 958.5/958.4MB downloaded


In [5]:
# importing data files
df1 = pd.read_csv('/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/1. Top City for Immigrants/1. Immigrant Cities.csv',index_col=0)
df2 = pd.read_csv('/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/1. Top City for Immigrants/2. Profession by Cities.csv',index_col=0)
df3 = pd.read_csv('/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/1. Top City for Immigrants/3. Immigrant Group Data.csv',index_col=0)
df4 = pd.read_csv('/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/1. Top City for Immigrants/4. State Migration Data.csv')

In [6]:
# merging the dataframes to one dataframe
df_merge = df1.merge(df3, left_index = True, right_index = True, how = 'left')
df_merge = df_merge.merge(df2,left_index = True, right_index = True, how = 'left')


In [7]:
merged_df = df_merge.merge(df4, how = 'left', left_on = 'STATE_x', right_on = 'STATE')
merged_df.set_index(df_merge.index, inplace = True)

In [8]:
# dropping columns not required for the analysis and creating a final dataframe
columns_to_drop = ['STATE_y','H_MEDIAN','A_MEDIAN','FOREIGN_BORN_POPULATION','STATE']
df = merged_df.drop(columns = columns_to_drop)

In [9]:
# preprocessing data format
df['TOT_EMP'] = pd.to_numeric(df['TOT_EMP'].str.replace(',', '').str.replace('*', ''), errors='coerce')
df['H_MEAN'] = pd.to_numeric(df['H_MEAN'].str.replace(',', '').str.replace('*', ''), errors='coerce')
df['A_MEAN'] = pd.to_numeric(df['A_MEAN'].str.replace(',', '').str.replace('*', ''), errors='coerce')
df['TOTAL_POPULATION'] = pd.to_numeric(df['TOTAL_POPULATION'].str.replace(',', '').str.replace('*', ''), errors='coerce')

df['TOT_EMP'] = df['TOT_EMP'].fillna(0)
df['H_MEAN'] = df['H_MEAN'].fillna(0)
df['A_MEAN'] = df['A_MEAN'].fillna(0)
df['TOTAL_POPULATION'] = df['TOTAL_POPULATION'].fillna(0)

df['TOT_EMP'] = df['TOT_EMP'].astype('int64')
df['H_MEAN'] = df['H_MEAN'].astype('float64')
df['A_MEAN'] = df['A_MEAN'].astype('float64')
df['TOTAL_POPULATION'] = df['TOTAL_POPULATION'].astype('float64')

<ipython-input-9-c8829d3e2550>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['TOT_EMP'] = pd.to_numeric(df['TOT_EMP'].str.replace(',', '').str.replace('*', ''), errors='coerce')
<ipython-input-9-c8829d3e2550>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['H_MEAN'] = pd.to_numeric(df['H_MEAN'].str.replace(',', '').str.replace('*', ''), errors='coerce')
<ipython-input-9-c8829d3e2550>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['A_MEAN'] = pd.to_numeric(df['A_MEAN'].str.replace(',', '').str.replace('*', 

In [10]:
def get_pos_tag(token):
    pos_tag = nltk.pos_tag([token])[0][1]
    return pos_tag

# Function to tokenize, lemmatize with POS tagging, and remove stop words from a text
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_tokens = []
    for token in tokens:
        pos_tag = get_pos_tag(token.lower())
        # Remove specific words and symbols
        if token.lower() not in ['general ','other', 'all', 'except', 'nonrestaurant', '/', '--']:
            if token.lower() not in stop_words:
                if pos_tag.startswith('V'):  # Verb
                    lemmatized_tokens.append(lemmatizer.lemmatize(token.lower(), pos='v'))
                elif pos_tag.startswith('N'):  # Noun
                    lemmatized_tokens.append(lemmatizer.lemmatize(token.lower(), pos='n'))
                else:
                    lemmatized_tokens.append(token.lower())
    return lemmatized_tokens

In [11]:
# converting text data into tags
df['PROFESSION'] = df['PROFESSION'].apply(lambda x: x.replace(',', ''))
df['PREPROCESSED_PROFESSION'] = df['PROFESSION'].apply(preprocess_text)
df['TAGS'] = df['PREPROCESSED_PROFESSION'].apply(set)

In [12]:
def convert_set_to_str(s):
    return str(list(s))

# Function to find similar words for a given word
def find_similar_words(word, threshold=0.75, topn=5):
    if word in model.key_to_index:
        query_vector = model.get_vector(word)
        similar_words = model.similar_by_vector(query_vector, topn=topn)
        similar_words = [word for word, score in similar_words if score >= threshold]
        return similar_words
    else:
        return []

# Expand the TAGS with similar words
def expand_tags_with_similar_words(tags_set):
    expanded_tags = set(tags_set)
    for word in tags_set:
        similar_words = find_similar_words(word)
        expanded_tags.update(similar_words)
    return expanded_tags


In [13]:
#df['EXPANDED_TAGS'] = df['TAGS']
df['EXPANDED_TAGS'] = df['TAGS'].apply(expand_tags_with_similar_words)

In [14]:
# Convert sets to string representation
df['TAGS'] = df['TAGS'].apply(convert_set_to_str)
df['EXPANDED_TAGS'] = df['EXPANDED_TAGS'].apply(convert_set_to_str)

In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

column_to_scale = df[['DIFFERENT_STATE_MOVE_IN']]

scaled_data = scaler.fit_transform(column_to_scale)

scaled_df = pd.DataFrame(scaled_data, columns=['DIFFERENT_STATE_MOVE_IN'], index=column_to_scale.index)

df['DIFFERENT_STATE_MOVE_IN'] = scaled_df['DIFFERENT_STATE_MOVE_IN']

In [16]:
df.to_pickle('/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/1. Top City for Immigrants/tagged_data.pkl')

In [17]:
df.head()

,STATE_x,TOTAL_POPULATION,SAFETY_INDEX,COST_OF_LIVING_INDEX,WINTER_COLDEST_TEMP,SUMMER_HOTTEST_TEMP,BEST_SUBURBS,IMAGE_LINK,Mexico,China,...,Nepal,Other Countries,PROFESSION,TOT_EMP,H_MEAN,A_MEAN,DIFFERENT_STATE_MOVE_IN,PREPROCESSED_PROFESSION,TAGS,EXPANDED_TAGS
CITY,,,,,,,,,,,,,,,,,,,,,
Atlanta,Georgia,540932.0,70,100,43,80,https://www.niche.com/places-to-live/search/be...,https://cdn.britannica.com/27/64827-050-0E5634...,137975,37052,...,3224,234520,Chief Executives,7360,116.85,243040.0,0.396707,"[chief, executive]","['executive', 'chief']","['exective', 'vice-chief', 'executive', 'chief..."
Atlanta,Georgia,540932.0,70,100,43,80,https://www.niche.com/places-to-live/search/be...,https://cdn.britannica.com/27/64827-050-0E5634...,137975,37052,...,3224,234520,General and Operations Managers,65150,59.75,124290.0,0.396707,"[general, operation, manager]","['operation', 'manager', 'general']","['general', 'operations', 'operation', 'manage..."
Atlanta,Georgia,540932.0,70,100,43,80,https://www.niche.com/places-to-live/search/be...,https://cdn.britannica.com/27/64827-050-0E5634...,137975,37052,...,3224,234520,Legislators,0,0.00,56810.0,0.396707,[legislator],['legislator'],"['lawmaker', 'legislator', 'Legislator']"
Atlanta,Georgia,540932.0,70,100,43,80,https://www.niche.com/places-to-live/search/be...,https://cdn.britannica.com/27/64827-050-0E5634...,137975,37052,...,3224,234520,Advertising and Promotions Managers,370,74.61,155180.0,0.396707,"[advertising, promotion, manager]","['promotion', 'manager', 'advertising']","['advertising-', 'adverting', 'manager', 'adve..."
Atlanta,Georgia,540932.0,70,100,43,80,https://www.niche.com/places-to-live/search/be...,https://cdn.britannica.com/27/64827-050-0E5634...,137975,37052,...,3224,234520,Marketing Managers,7140,73.79,153480.0,0.396707,"[marketing, manager]","['marketing', 'manager']","['direct-marketing', 'marketing-', 'mass-marke..."
